The following are algorithms about assembly

In [1]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

build overlap graph by finding overlapping sequence

In [4]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
def overlap_all_pairs(seq,length):
    #build dictionary
    kmer_dict = {}
    for s in seq:
        for i in range(len(s)-length+1):
            start = i
            end = i+length
            kmer = s[start:end]
            if kmer in kmer_dict:
                kmer_dict[kmer].append(s)
            else:
                kmer_dict[kmer] = [s]
    #print kmer_dict
    #find overlap
    match_pairs = []
    for a in seq:
        suffix = a[-length:]
        match = kmer_dict[suffix]
        for b in match:
            if b==a:
                continue
            ol = overlap(a, b, min_length=length)
            if ol!=0:
                match_pairs.append((a,b))
    return match_pairs

In [5]:
phi_sequence,qualities = readFastq('ERR266411_1.for_asm.fastq')
pairs = overlap_all_pairs(phi_sequence, 30)
print(len(pairs))

904746


Greedy assembly algorithm

In [10]:
import itertools

In [16]:
def pick_maximal_overlap(reads, k,kmer_dict):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    reada, readb = None, None
    best_olen = 0
    for a, b in itertools.permutations(reads, 2):
        suffix = a[-k:]
        match = kmer_dict[suffix]
        if b not in match:
            continue
        olen = overlap(a, b, min_length=k)
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen
def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    kmer_dict = {}
    for s in reads:
        for i in range(len(s)-k+1):
            start = i
            end = i+k
            kmer = s[start:end]
            if kmer in kmer_dict:
                kmer_dict[kmer].append(s)
            else:
                kmer_dict[kmer] = [s]
    print (len(kmer_dict))
    read_a, read_b, olen = pick_maximal_overlap(reads, k,kmer_dict)
    while olen > 0:
        if len(reads)%10==0:
            print len(reads)
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k,kmer_dict)
    return ''.join(reads)

In [ ]:
virus_sequence,qualities = readFastq('ads1_week4_reads.fq')
assem = greedy_scs(virus_sequence, 10)
print(len(assem))

In [28]:
assem = greedy_scs(virus_sequence, 10)
print(len(assem))

15668
15894
